In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# From Data file we will import the finall_data.xlsx

df = pd.read_excel(r'C:\Users\Dell\OneDrive\Desktop\advance web scraping\Nakri_data_set\Skill_Gap_Analysis_Tool\Data\Finall_data.xlsx')
df.info()

In [ ]:
# removed the use less columns  
df.drop(columns=['Unnamed: 0.1'	,'Unnamed: 0'	],inplace=True)
df.head()
df.columns

In [ ]:
temp_df = df.copy()

In [ ]:
temp_df = temp_df[temp_df['job_pay'] != 'Not disclosed']

In [ ]:
temp_df['starting_range'] = temp_df['job_pay'].str.split('-',expand=True)[0]
temp_df['ending_range'] = temp_df['job_pay'].str.split('-',expand=True)[1]
temp_df['starting_range'].value_counts()

In [ ]:
temp_df['starting_range'] = temp_df['job_pay'].str.split('-',expand=True)[0]
temp_df['ending_range'] = temp_df['job_pay'].str.split('-',expand=True)[1]
temp_df['ending_range'] = temp_df['ending_range'].fillna('No data')

# Create a new column 'bonus' with default value 0
temp_df['bonus'] = 0

# Find rows where 'ending_range' contains '(' and set 'bonus' to 1 for those rows
temp_df.loc[temp_df['ending_range'].str.contains(r'\('), 'bonus'] = 1

indexes = temp_df[temp_df['ending_range'].str.contains(r'\(')].index

# Split 'ending_range' at '(' and take the first part
split_values = temp_df.loc[indexes, 'ending_range'].str.split(r'\(', expand=True)[0]

# Update the DataFrame with the split values
temp_df.loc[indexes, 'ending_range'] = split_values

# temp_df['ending_range'].str.replace(r'.*\(([^:]+): ([^\)]+)\).*', r'\0', regex=True)

In [ ]:
# For rows where 'starting_range' is 'No data' and 'Company Size' is 'Very Large', set 'starting_range' to 7
temp_df.loc[(temp_df['ending_range'] == 'No data') & (temp_df['Company Size'] == 'Very Large'), 'ending_range'] = 1200000
temp_df.loc[(temp_df['ending_range'] == 'No data') & (temp_df['Company Size'] == 'Large'), 'ending_range'] = 1000000
temp_df.loc[(temp_df['ending_range'] == 'No data') & (temp_df['Company Size'] == 'Small'), 'ending_range'] = 500000
temp_df.loc[(temp_df['ending_range'] == 'No data') & (temp_df['Company Size'] == 'Very Small'), 'ending_range'] = 300000
temp_df.loc[(temp_df['job_pay'] == 'No data') & (temp_df['Company Size'] == 'Very Large'), 'starting_range'] = 900000
temp_df.loc[(temp_df['job_pay'] == 'No data') & (temp_df['Company Size'] == 'Large'), 'starting_range'] = 700000
temp_df.loc[(temp_df['job_pay'] == 'No data') & (temp_df['Company Size'] == 'Small'), 'starting_range'] = 500000
temp_df.loc[(temp_df['job_pay'] == 'No data') & (temp_df['Company Size'] == 'Very Small'), 'starting_range'] = 200000

# For rows where 'starting_range' is 'No data' and 'Company Size' is 'Large', set 'starting_range' to 5
temp_df.loc[(temp_df['ending_range'] == 'No data') & (temp_df['Company Size'] == 'Medium'), 'ending_range'] = 700000
temp_df.loc[(temp_df['job_pay'] == 'No data') & (temp_df['Company Size'] == 'Medium'), 'starting_range'] = 500000

# For rows where 'starting_range' is 'No data' and 'Company Size' is 'Small', set 'starting_range' to 3


In [ ]:
temp_df

In [ ]:


def process_value(value):
    if isinstance(value, str):
        if 'Cr' in value:
            return float(value.replace(' Cr', '')) * 10000000
        elif ',' in value:
            return float(value.replace(',', '')) 
        else:
            return float(value) * 100000
    else:
        return value

# Applying the function to the column
temp_df['ending_range'] = temp_df['ending_range'].apply(process_value)
# def process_value(value):
#     if isinstance(value, str):
#         if 'Cr' in value:
#             return float(value.replace(' Cr', '')) * 10000000
#         elif ',' in value:
#             return float(value.replace(',', '')) 
#         else:
#             return float(value) * 100000
#     else:
#         return value
    



# # Applying the function to the column
temp_df['starting_range'] = temp_df['starting_range'].str.replace(' Lacs', '')
temp_df['starting_range'] = temp_df['starting_range'].apply(process_value)

In [ ]:
# temp_df_df

In [ ]:
temp_df.info()

In [ ]:
# temp_df['starting_range'].astype(int)
# temp_df['ending_range'].astype(int)
temp_df.drop(columns=['Description'],inplace=True)

In [ ]:
temp_df.info()

In [ ]:
# temp_df.drop(temp_df['skill_7'].isnull().index,inplace=True)     
temp_df['normalized_salary'] = (temp_df['starting_range'] + temp_df['ending_range']) / 2
   

In [ ]:
# temp_df.drop(columns=['skill_10'], inplace=True)
temp_df.info()

In [ ]:
temp_df[temp_df['starting_range'].isna()]

In [ ]:
temp_df.to_excel('with_price.xlsx')

In [ ]:
temp_df_protect =  temp_df.copy()

In [ ]:
df = temp_df.copy() 
# Create a list of unique skills
# Example: Define skill sets for job categories
unique_skills = set()
for i in range(1, 9):
    skill_col = f'skill_{i}'
    unique_skills.update(df[skill_col].dropna().unique())

# Create binary columns for each unique skill
for skill in unique_skills:
    df[skill] = 0

# Populate the binary columns
for i in range(1, 9):
    skill_col = f'skill_{i}'
    for skill in unique_skills:
        df[skill] = df[skill] | (df[skill_col] == skill).astype(int)

# Step 2: Skill Count
# Count non-null skill entries per row
df['total_skills'] = df[[f'skill_{i}' for i in range(1, 9)]].notnull().sum(axis=1)

# Step 3: Binary Encoding for Job Categories
# Get unique job categories
unique_job_categories = df['Job Category'].unique()

# Create binary columns for each job category
for category in unique_job_categories:
    df[category] = df['Job Category'] == category

# Convert boolean columns to integers (0 or 1)
df[unique_job_categories] = df[unique_job_categories].astype(int)

# Drop the original skill columns and 'Job Category' column if no longer needed
df.drop(columns=[f'skill_{i}' for i in range(1, 9)] + ['Job Category'], inplace=True)

df.head()


In [ ]:
df.columns

temp = pd.DataFrame(index = df.columns)
# df.drop(columns=['Company'],inplace=True)
temp['data_type'] = df.dtypes
temp['unique_value'] = df.nunique()
temp['corr_values'] = df.corr()['normalized_salary']
temp

In [ ]:
df[]

In [ ]:
df['Data and Analysis'].value_counts()

In [ ]:
df['Job Category']

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming the DataFrame `df` is already prepared as shown in your message

# Correlation Analysis
correlation_matrix = df.corr()

# Select salary-related columns to analyze
salary_corr_starting = correlation_matrix['starting_range'].sort_values(ascending=False)
salary_corr_ending = correlation_matrix['ending_range'].sort_values(ascending=False)

# Display top correlations with starting salary
print("Top correlations with starting salary:")
print(salary_corr_starting.head(10))

# Display top correlations with ending salary
print("Top correlations with ending salary:")
print(salary_corr_ending.head(10))

# Visualization of the correlation matrix for the top features
top_features_starting = salary_corr_starting.index[:10]
top_features_ending = salary_corr_ending.index[:10]

# Combine the unique top features for a comprehensive correlation matrix
top_features = list(set(top_features_starting).union(set(top_features_ending)))

# Heatmap for top features correlation
plt.figure(figsize=(12, 10))
sns.heatmap(df[top_features].corr(), annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix of Top Features')
plt.show()

# Visualization: Boxplot of starting salary by total number of skills
plt.figure(figsize=(14, 7))
sns.boxplot(x='total_skills', y='starting_range', data=df)
plt.xticks(rotation=90)
plt.title('Starting Salary Range by Total Number of Skills')
plt.show()

# Visualization: Boxplot of ending salary by job categories
plt.figure(figsize=(14, 7))
sns.boxplot(x='Data and Analysis', y='ending_range', data=df)
plt.xticks(rotation=90)
plt.title('Ending Salary Range by Job Category: Data and Analysis')
plt.show()



In [ ]:
# temp_df = temp_df[~temp_df['skill_3'].isna()]

In [ ]:
temp_df

In [ ]:
# # Assume we have a dictionary of skill importance
# skill_importance = {'Python': 5, 'Java': 4, 'SQL': 3, 'Machine Learning': 5, 'Deep Learning': 5,}

# # Create a new column for skill score 
# df['skill_score'] = 0

# # Calculate the skill score for each job
# for i in range(1, 11):
#     skill_col = f'skill_{i}'
#     df['skill_score'] += df[skill_col].map(skill_importance).fillna(0)

# print(df[['Job_role', 'skill_score']].head())


In [ ]:
temp = pd.DataFrame()

temp['unique_skill'] = df.nunique()
temp['null_value'] = df.isnull().sum()
temp['null_value'] = df.isnull().sum()
temp

In [ ]:
import matplotlib